In [ ]:
## Import some helpful libraries
print("    Version control\n------------------------")
import os, fnmatch, random, math, sys, datetime
from pathlib import Path
import numpy as np;              print("Numpy\t\t", np.__version__)
import matplotlib as mpl;        print("matplotlib\t", mpl.__version__)
import matplotlib.pyplot as plt
import pandas as pd;             print("Pandas\t\t {}".format(pd.__version__))
import h5py;                     print("H5py\t\t {}".format(h5py.__version__))
import tensorflow as tf;         print("TensorFlow\t {}".format(tf.__version__))
import keras;                    print("Keras\t\t {}".format(keras.__version__))
import tensorflow.keras.backend as K
from tensorflow.keras import models, Input, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, Conv3D, MaxPooling3D, UpSampling3D, ZeroPadding3D
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer, InputSpec
from keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [ ]:
## Load all data
sample_name = np.loadtxt(('../Data/data_random_1/raw_data/all_sample_name.csv'), dtype=str, delimiter=",")
all_data = np.load('../Data/data_random_1/raw_data/all.npy').reshape(124,182,218,182,1)
print('all_data shape is {}'.format(all_data.shape))

In [ ]:
## Padding datasets
all_pad_data = (ZeroPadding3D(padding=(5,19,5))(all_data)).numpy()
print('all_padded_data shape is {}'.format(all_pad_data.shape))

In [ ]:
## Convolutional AutoEncoder Model Designe:

BATCH_SIZE = 1
EPOCHS = 1500

IMAGE_HEIGHT = all_pad_data.shape[1]
IMAGE_WIDTH  = all_pad_data.shape[2]
IMAGE_DEPTH  = all_pad_data.shape[3]
data_shape = [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
input_shape = [BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
print("Padded data shape: {}\n".format(input_shape))

## Encoder
input_img = Input(shape=(192, 256, 192, 1), name='Input')
x1 = Conv3D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv1')(input_img)
x1 = BatchNormalization(name='BN_Conv1')(x1)
x2 = Conv3D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv2')(x1)
x2 = BatchNormalization(name='BN_Conv2')(x2)
x3 = Conv3D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv3')(x2)
x3 = BatchNormalization(name='BN_Conv3')(x3)
x4 = Conv3D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv4')(x3)
x4 = BatchNormalization(name='BN_Conv4')(x4)

## Latent Features
shape_before_flattening = tf.keras.backend.int_shape(x4)
x_LF = Flatten(name='LF')(x4)
encoded = x_LF
x5 = Reshape(shape_before_flattening[1:], name='UnFlat')(encoded)

## Decoder
x5 = Conv3D(filters=16, kernel_size=3, padding='same', activation='relu', name='DeConv3')(x5)
x5 = BatchNormalization(name='BN_DeConv3')(x5)
x5 = UpSampling3D(size=(2, 2, 2), name='UpSampling3')(x5)
x6 = Conv3D(filters=32, kernel_size=3, padding='same', activation='relu', name='DeConv4')(x5)
x6 = BatchNormalization(name='BN_DeConv4')(x6)
x6 = UpSampling3D(size=(2, 2, 2), name='UpSampling4')(x6)
x7 = Conv3D(filters=64, kernel_size=3, padding='same', activation='relu', name='DeConv5')(x6)
x7 = BatchNormalization(name='BN_DeConv5')(x7)
x7 = UpSampling3D(size=(2, 2, 2), name='UpSampling5')(x7)
x8 = Conv3D(filters=128, kernel_size=3, padding='same', activation='relu', name='DeConv6')(x7)
x8 = BatchNormalization(name='BN_DeConv6')(x8)
x8 = UpSampling3D(size=(2, 2, 2), name='UpSampling6')(x8)
decoded = Conv3D(filters=1, kernel_size=3, padding='same', activation='sigmoid', name='Output')(x8)

model_CAE = Model(inputs=input_img, outputs=decoded, name='AutoEncoder')
## optimizer=rmsprop, sgd
model_CAE.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_CAE.summary()

In [ ]:
## Start time:
from datetime import datetime
start_time = datetime.now().strftime("%Y_%m_%d___%H_%M"); print("\nStart Time =", start_time, "\n")

## Model Fit
# model_CAE.load_weights('./Weights/all___2021_01_07___11_42.hdf5', by_name=True)  # epochs 1001_1500
tb_callback = TensorBoard(('./Logs/all___' + start_time), histogram_freq=1)
model_CAE.fit(all_pad_data, all_pad_data, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, callbacks=[tb_callback], verbose=1)
# model_CAE.fit(train_pad_data, train_pad_data, validation_data=(valid_pad_data, valid_pad_data), epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, callbacks=[tb_callback, model_checkpoint_callback], verbose=1)
model_CAE.save_weights('./Weights/all___' + start_time + '.hdf5')

## End time:
from datetime import datetime
end_time = datetime.now().strftime("%Y_%m_%d___%H_%M"); print("\nEnd Time =", end_time)